In [1]:
import pandas as pd
import numpy as np
import pprint as pp

# Functions

In [2]:
def trova_voti_lista_plurinom(df, lista):
    
    sub = df[df['LISTA'] == lista]
    sub = sub[['COLLEGIOPLURINOMINALE 2022', 
               'VOTI_LISTA']].groupby('COLLEGIOPLURINOMINALE 2022').sum().reset_index()
    
    sub.columns = ['COLLEGIOPLURINOMINALE 2022', 'VOTI_LISTA_{}'.format(lista)]
    
    return sub


def get_metadata_plurinom_camera(path):
    
    meta = pd.read_excel(path, sheet_name = 'Table 8')
    
    # Fix columns 
    meta.columns = meta.iloc[0].tolist()
    
    # Drop first row
    meta = meta.drop(axis = 0, index = 0)

    # Fill NA for CIRCOSCRIZIONI
    meta['CIRCOSCRIZIONI'] = meta['CIRCOSCRIZIONI'].fillna(method = 'ffill')
    
    # Remove last columns who is an NA
    meta = meta.iloc[:-1, :-1]

    # Further remove NAs
    meta = meta.dropna()
    
    return meta

def get_metadata_plurinom_senato(path):
    
    meta = pd.read_excel(path, sheet_name = 'Table 9')
    
    # Fix columns 
    meta.columns = meta.iloc[0].tolist()
    
    # Drop first row
    meta = meta.drop(axis = 0, index = 0)

    # Fill NA for CIRCOSCRIZIONI
    meta['CIRCOSCRIZIONI'] = meta['CIRCOSCRIZIONI'].fillna(method = 'ffill')
    
    # Remove last columns who is an NA
    meta = meta.iloc[:-4, :-1]
    
    # Fix seggi plurinominali
    meta['Seggi plurinominali'] = meta['Seggi plurinominali'].replace('-', np.nan)

    # Further remove NAs
    meta = meta.dropna()
    
    return meta

# Analisi Camera

In [3]:
# Read Data
camera = pd.read_csv('camera-20180304.txt', sep = ";", encoding = "ISO-8859-1")
camera.rename(columns = {'COLLEGIOUNINOMINALE': 'COLLEGIOUNINOMINALE 2018',
                         'COLLEGIOPLURINOMINALE': 'COLLEGIOPLURINOMINALE 2018'
                        }, inplace = True)

In [4]:
# Read Data - Mapping nuovi collegi
mapping_collegi = pd.read_excel('mapping_collegi.xlsx')
mapping_collegi.head()

,CIRCOSCRIZIONE,COLLEGIOPLURINOMINALE 2018,COLLEGIOUNINOMINALE 2018,COLLEGIOPLURINOMINALE 2022
0,ABRUZZO,ABRUZZO - 01,03 - PESCARA,ABRUZZO - 01
1,ABRUZZO,ABRUZZO - 01,04 - CHIETI,ABRUZZO - 01
2,ABRUZZO,ABRUZZO - 01,05 - VASTO,ABRUZZO - 01
3,ABRUZZO,ABRUZZO - 02,01 - L'AQUILA,ABRUZZO - 01
4,ABRUZZO,ABRUZZO - 02,02 - TERAMO,ABRUZZO - 01


In [5]:
mapping_collegi[mapping_collegi['CIRCOSCRIZIONE'] == 'TOSCANA']

,CIRCOSCRIZIONE,COLLEGIOPLURINOMINALE 2018,COLLEGIOUNINOMINALE 2018,COLLEGIOPLURINOMINALE 2022
190,TOSCANA,TOSCANA - 01,08 - MASSA,TOSCANA - 01
191,TOSCANA,TOSCANA - 01,09 - LUCCA,TOSCANA - 01
192,TOSCANA,TOSCANA - 01,05 - PRATO,TOSCANA - 01
193,TOSCANA,TOSCANA - 01,06 - PISTOIA,TOSCANA - 01
194,TOSCANA,TOSCANA - 02,11 - POGGIBONSI,TOSCANA - 02
195,TOSCANA,TOSCANA - 04,07 - AREZZO,TOSCANA - 02
196,TOSCANA,TOSCANA - 04,12 - SIENA,TOSCANA - 02
197,TOSCANA,TOSCANA - 04,14 - GROSSETO,TOSCANA - 02
198,TOSCANA,TOSCANA - 02,13 - LIVORNO,TOSCANA - 02
199,TOSCANA,TOSCANA - 02,10 - PISA,TOSCANA - 03


In [6]:
camera = pd.merge(camera,
         mapping_collegi[['COLLEGIOUNINOMINALE 2018', 'COLLEGIOPLURINOMINALE 2022']],
         on = 'COLLEGIOUNINOMINALE 2018',
         how = 'left')

In [7]:
# Crea tabella elettori e votanti collegio plurinominale
plurinom = camera[['COLLEGIOPLURINOMINALE 2022',
                   'COLLEGIOUNINOMINALE 2018',
                   'COMUNE', 
                   'ELETTORI',
                   'VOTANTI']].drop_duplicates()

plurinom = plurinom.groupby('COLLEGIOPLURINOMINALE 2022').agg({'ELETTORI': np.sum,
                                               'VOTANTI': np.sum
                                              })

plurinom.columns = ['ELETTORI_PLURINOMINALE', 'VOTANTI_PLURINOMINALE']

plurinom = plurinom.reset_index()

#.agg({'ELETTORI': np.sum(),'VOTANTI': np.sum()})

In [8]:
path = 'commissione-collegi-elettorali_relazione_13-novembre-2020-con-errata-corrige.xlsx'
meta_camera = get_metadata_plurinom_camera(path)

meta_camera.head()

,CIRCOSCRIZIONI,Collegi plurinominali,Popolazione residente,Seggi totali,Popolazione\nmedia per seggio,Seggi uninominali,Popolazione\nmedia\nper seggio uninominale,Seggi plurinominali
2,Piemonte 1,Piemonte 1 - 01,1241100,8,155137,3,413700,5
3,Piemonte 1,Piemonte 1 - 02,1006680,7,143811,2,503340,5
5,Piemonte 2,Piemonte 2 - 01,884956,6,147492,2,442478,4
6,Piemonte 2,Piemonte 2 - 02,1231180,8,153897,3,410393,5
8,Lombardia 1,Lombardia 1 - 01,1994822,13,153447,5,398964,8


In [25]:
meta_camera['Seggi plurinominali'].sum()

490

In [26]:
meta_camera['Seggi uninominali'].sum()

293

In [9]:
# Aggiungi metatadati collegi plurinominali

path = 'commissione-collegi-elettorali_relazione_13-novembre-2020-con-errata-corrige.xlsx'
meta_camera = get_metadata_plurinom_camera(path)
meta_camera.rename(columns={"Collegi plurinominali":"COLLEGIOPLURINOMINALE 2022"}, inplace = True)
meta_camera['COLLEGIOPLURINOMINALE 2022'] = [x.upper() for x in meta_camera['COLLEGIOPLURINOMINALE 2022'].astype(str)]

plurinom = pd.merge(plurinom, meta_camera, on = "COLLEGIOPLURINOMINALE 2022", how = 'left')

In [10]:
# Trova percentuali di lista

sub_pap = trova_voti_lista_plurinom(camera, 'POTERE AL POPOLO!')
sub_sin_riv = trova_voti_lista_plurinom(camera, 'PER UNA SINISTRA RIVOLUZIONARIA')
sub_part_com = trova_voti_lista_plurinom(camera, 'PARTITO COMUNISTA')
sub_part_lib = trova_voti_lista_plurinom(camera, 'LIBERI E UGUALI')
sub_part_m5s = trova_voti_lista_plurinom(camera, 'MOVIMENTO 5 STELLE')

In [11]:
rep = pd.merge(plurinom, sub_pap, on = 'COLLEGIOPLURINOMINALE 2022', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100


In [12]:
rep = pd.merge(plurinom, sub_pap, on = 'COLLEGIOPLURINOMINALE 2022', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_sin_riv, on = 'COLLEGIOPLURINOMINALE 2022', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_com, on = 'COLLEGIOPLURINOMINALE 2022', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_m5s, on = 'COLLEGIOPLURINOMINALE 2022', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_lib, on = 'COLLEGIOPLURINOMINALE 2022', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep_camera = rep.sort_values('VOTI_LISTA_POTERE AL POPOLO!', ascending = False).fillna(0)

# Analisi Senato

In [14]:
# Read Data
senato = pd.read_csv('senato-20180304.txt', sep = ";", encoding = "ISO-8859-1")
senato.rename(columns = {'COLLEGIOUNINOMINALE': 'COLLEGIOUNINOMINALE 2018',
                         'COLLEGIOPLURINOMINALE': 'COLLEGIOPLURINOMINALE 2018'
                        }, inplace = True)

In [15]:
# Read Data - Mapping nuovi collegi senato
mapping_collegi = pd.read_excel('mapping_collegi_senato.xlsx')
mapping_collegi.head()

,REGIONE,COLLEGIOPLURINOMINALE 2018,COLLEGIOUNINOMINALE 2018,COLLEGIOPLURINOMINALE 2022
0,PIEMONTE,PIEMONTE - 01,01 - SETTIMO TORINESE,PIEMONTE - 01
1,PIEMONTE,PIEMONTE - 01,02 - MONCALIERI,PIEMONTE - 01
2,PIEMONTE,PIEMONTE - 01,03 - TORINO - COLLEGNO,PIEMONTE - 01
3,PIEMONTE,PIEMONTE - 01,04 - TORINO - ZONA STATISTICA 61,PIEMONTE - 01
4,PIEMONTE,PIEMONTE - 02,05 - NOVARA,PIEMONTE - 02


In [16]:
mapping_collegi[mapping_collegi['REGIONE'] == 'CAMPANIA']

,REGIONE,COLLEGIOPLURINOMINALE 2018,COLLEGIOUNINOMINALE 2018,COLLEGIOPLURINOMINALE 2022
79,CAMPANIA,CAMPANIA - 02,04 - GIUGLIANO IN CAMPANIA,CAMPANIA - 01
80,CAMPANIA,CAMPANIA - 02,06 - CASORIA,CAMPANIA - 01
81,CAMPANIA,CAMPANIA - 02,07 - NAPOLI - CIRCOSCRIZIONE 7 SAN CARLO ALL'A...,CAMPANIA - 01
82,CAMPANIA,CAMPANIA - 02,08 - NAPOLI - CIRCOSCRIZIONE 19 FUORIGROTTA,CAMPANIA - 01
83,CAMPANIA,CAMPANIA - 03,05 - PORTICI,CAMPANIA - 01
84,CAMPANIA,CAMPANIA - 03,09 - TORRE DEL GRECO,CAMPANIA - 01
85,CAMPANIA,CAMPANIA - 01,01 - BENEVENTO,CAMPANIA - 02
86,CAMPANIA,CAMPANIA - 01,02 - CASERTA,CAMPANIA - 02
87,CAMPANIA,CAMPANIA - 01,03 - AVELLINO,CAMPANIA - 02
88,CAMPANIA,CAMPANIA - 03,10 - SALERNO,CAMPANIA - 02


In [17]:
senato = pd.merge(senato,
         mapping_collegi[['COLLEGIOUNINOMINALE 2018', 'COLLEGIOPLURINOMINALE 2022']],
         on = 'COLLEGIOUNINOMINALE 2018',
         how = 'left')

In [18]:
# Crea tabella elettori e votanti collegio plurinominale
plurinom = senato[['COLLEGIOPLURINOMINALE 2022',
                   'COLLEGIOUNINOMINALE 2018',
                   'COMUNE', 
                   'ELETTORI',
                   'VOTANTI']].drop_duplicates()

plurinom = plurinom.groupby('COLLEGIOPLURINOMINALE 2022').agg({'ELETTORI': np.sum,
                                               'VOTANTI': np.sum
                                              })

plurinom.columns = ['ELETTORI_PLURINOMINALE', 'VOTANTI_PLURINOMINALE']# Crea tabella elettori e votanti collegio plurinominale
plurinom = senato[['COLLEGIOPLURINOMINALE 2022',
                   'COLLEGIOUNINOMINALE 2018',
                   'COMUNE', 
                   'ELETTORI',
                   'VOTANTI']].drop_duplicates()

plurinom = plurinom.groupby('COLLEGIOPLURINOMINALE 2022').agg({'ELETTORI': np.sum,
                                               'VOTANTI': np.sum
                                              })

plurinom.columns = ['ELETTORI_PLURINOMINALE', 'VOTANTI_PLURINOMINALE']

In [19]:
# Aggiungi metatadati collegi plurinominali

path = 'commissione-collegi-elettorali_relazione_13-novembre-2020-con-errata-corrige.xlsx'
meta_senato = get_metadata_plurinom_senato(path)
meta_senato.rename(columns={"Collegi plurinominali":"COLLEGIOPLURINOMINALE 2022"}, inplace = True)
meta_senato['COLLEGIOPLURINOMINALE 2022'] = [x.upper() for x in meta_senato['COLLEGIOPLURINOMINALE 2022'].astype(str)]

plurinom = pd.merge(plurinom, meta_senato, on = "COLLEGIOPLURINOMINALE 2022", how = 'left')

In [20]:
sub_pap = trova_voti_lista_plurinom(senato, 'POTERE AL POPOLO!')
sub_sin_riv = trova_voti_lista_plurinom(senato, 'PER UNA SINISTRA RIVOLUZIONARIA')
sub_part_com = trova_voti_lista_plurinom(senato, 'PARTITO COMUNISTA')
sub_part_lib = trova_voti_lista_plurinom(senato, 'LIBERI E UGUALI')
sub_part_m5s = trova_voti_lista_plurinom(senato, 'MOVIMENTO 5 STELLE')

In [21]:
rep = pd.merge(plurinom, sub_pap, on = 'COLLEGIOPLURINOMINALE 2022', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_sin_riv, on = 'COLLEGIOPLURINOMINALE 2022', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_com, on = 'COLLEGIOPLURINOMINALE 2022', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_m5s, on = 'COLLEGIOPLURINOMINALE 2022', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep = pd.merge(rep, sub_part_lib, on = 'COLLEGIOPLURINOMINALE 2022', how = 'left')
rep[f"{rep.columns[-1]}_PERC"] =  (rep[rep.columns[-1]] / rep['VOTANTI_PLURINOMINALE'])*100

rep_senato = rep.sort_values('VOTI_LISTA_POTERE AL POPOLO!_PERC', ascending = False).fillna(0)

# Scenario Analysis



In [22]:
# Scenario 1 - PAP si allea con forze comuniste che si assume mantengano stessi risultati del 2018. 
# Anche PAP mantiene gli stessi risultati del 2018.

rep_camera['Scenario 1'] = (rep_camera['VOTI_LISTA_POTERE AL POPOLO!_PERC'] + 
                            rep_camera['VOTI_LISTA_PARTITO COMUNISTA_PERC'] +
                            rep_camera['VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC'])

rep_senato['Scenario 1'] = (rep_senato['VOTI_LISTA_POTERE AL POPOLO!_PERC'] + 
                            rep_senato['VOTI_LISTA_PARTITO COMUNISTA_PERC'] +
                            rep_senato['VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC'])

rep_camera.sort_values(by = 'Scenario 1', ascending = False)

,COLLEGIOPLURINOMINALE 2022,ELETTORI_PLURINOMINALE,VOTANTI_PLURINOMINALE,CIRCOSCRIZIONI,Popolazione residente,Seggi totali,Popolazione\nmedia per seggio,Seggi uninominali,Popolazione\nmedia\nper seggio uninominale,Seggi plurinominali,...,VOTI_LISTA_POTERE AL POPOLO!_PERC,VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA,VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC,VOTI_LISTA_PARTITO COMUNISTA,VOTI_LISTA_PARTITO COMUNISTA_PERC,VOTI_LISTA_MOVIMENTO 5 STELLE,VOTI_LISTA_MOVIMENTO 5 STELLE_PERC,VOTI_LISTA_LIBERI E UGUALI,VOTI_LISTA_LIBERI E UGUALI_PERC,Scenario 1
42,TOSCANA - 03,955731,757008,Toscana,1280561,8,160070,3,426854,5,...,2.240531,1296.0,0.171200,7009.0,0.925882,162042.0,21.405586,41713.0,5.510246,3.337613
41,TOSCANA - 02,1035692,803183,Toscana,1205308,8,150663,3,401769,5,...,1.722023,1612.0,0.200701,8843.0,1.100994,189828.0,23.634464,28330.0,3.527216,3.023719
11,LAZIO 1 - 01,1005501,720702,Lazio 1,1163634,8,145454,3,387878,5,...,1.884690,595.0,0.082558,3610.0,0.500901,190302.0,26.405088,29446.0,4.085739,2.468149
3,CAMPANIA 1 - 01,1216345,756959,Campania 1,1425848,9,158427,3,475283,6,...,2.210424,924.0,0.122067,0.0,0.000000,380025.0,50.204172,24058.0,3.178243,2.332491
40,TOSCANA - 01,849708,640768,Toscana,1186333,8,148291,3,395444,5,...,1.408778,0.0,0.000000,5501.0,0.858501,152757.0,23.839674,20646.0,3.222071,2.267279
12,LAZIO 1 - 02,1662264,1198415,Lazio 1,1258261,8,157282,3,419420,5,...,1.509243,1049.0,0.087532,6203.0,0.517600,372026.0,31.043170,43838.0,3.657998,2.114376
7,EMILIA-ROMAGNA - 01,930541,715695,Emilia-Romagna,1192490,8,149061,3,397497,5,...,1.077973,1159.0,0.161940,6186.0,0.864335,176684.0,24.687052,26078.0,3.643731,2.104248
44,UMBRIA - 01,672313,525978,Umbria,884268,6,147378,2,442134,4,...,1.211648,0.0,0.000000,4392.0,0.835016,133291.0,25.341554,14239.0,2.707147,2.046664
8,EMILIA-ROMAGNA - 02,1179159,930202,Emilia-Romagna,1698896,11,154445,4,424724,7,...,1.308748,2082.0,0.223822,4478.0,0.481401,231934.0,24.933724,48080.0,5.168770,2.013971
16,LIGURIA - 01,1229500,885066,Liguria,1570694,10,157069,4,392674,6,...,1.269623,1333.0,0.150610,4869.0,0.550128,247751.0,27.992376,35437.0,4.003882,1.970362


In [23]:
# Scenario 2 - PAP corre da sola e riesce a conquistare il 10% dei voti di liberi & uguali e il 5% dei 
# voti del movimento cinque stelle

rep_camera['Scenario 2'] = (rep_camera['VOTI_LISTA_POTERE AL POPOLO!_PERC'] + 
                            rep_camera['VOTI_LISTA_MOVIMENTO 5 STELLE_PERC'] * 0.05 +
                            rep_camera['VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC'] * 0.1
                           )

rep_senato['Scenario 2'] = (rep_senato['VOTI_LISTA_POTERE AL POPOLO!_PERC'] + 
                            rep_senato['VOTI_LISTA_MOVIMENTO 5 STELLE_PERC'] * 0.05 +
                            rep_senato['VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC'] * 0.1
                           )

rep_senato.sort_values(by = 'Scenario 2', ascending = False)

,COLLEGIOPLURINOMINALE 2022,ELETTORI_PLURINOMINALE,VOTANTI_PLURINOMINALE,CIRCOSCRIZIONI,Popolazione residente,Seggi totali,Popolazione\nmedia per seggio,Seggi uninominali,Popolazione\nmedia per seggio uninominale,Seggi plurinominali,...,VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA,VOTI_LISTA_PER UNA SINISTRA RIVOLUZIONARIA_PERC,VOTI_LISTA_PARTITO COMUNISTA,VOTI_LISTA_PARTITO COMUNISTA_PERC,VOTI_LISTA_MOVIMENTO 5 STELLE,VOTI_LISTA_MOVIMENTO 5 STELLE_PERC,VOTI_LISTA_LIBERI E UGUALI,VOTI_LISTA_LIBERI E UGUALI_PERC,Scenario 1,Scenario 2
2,CAMPANIA - 01,2139884,1391149,Campania,3275552,10,327555,4,818888,6.0,...,1356.0,0.097473,929.0,0.066779,691791.0,49.728031,34516.0,2.481115,1.688173,4.020069
7,LAZIO - 01,1620036,1167625,Lazio,2722036,9,302448,3,907345,6.0,...,1279.0,0.109539,5893.0,0.504700,311760.0,26.700353,51685.0,4.426507,2.520672,3.252406
3,CAMPANIA - 02,1948860,1383048,Campania,2491258,8,311407,3,830419,5.0,...,1791.0,0.129497,4201.0,0.303749,564472.0,40.813623,29615.0,2.141285,1.422221,3.042606
1,CALABRIA - 01,1825586,1191538,Calabria,1959050,6,326508,2,979525,4.0,...,1301.0,0.109187,5906.0,0.495662,476133.0,39.959531,37502.0,3.147361,1.581821,2.985868
14,MOLISE - 01,233710,166650,Molise,313660,2,156830,1,313660,1.0,...,0.0,0.000000,1163.0,0.697870,67295.0,40.381038,5337.0,3.202520,1.659166,2.980348
20,SICILIA - 02,1757007,1131345,Sicilia,2609466,8,326183,3,869822,5.0,...,0.0,0.000000,0.0,0.000000,512623.0,45.310935,26292.0,2.323960,0.620147,2.885694
0,ABRUZZO - 01,964353,723303,Abruzzo,1307309,4,326827,1,1307309,3.0,...,0.0,0.000000,4126.0,0.570439,260897.0,36.070222,16806.0,2.323508,1.648825,2.881897
17,PUGLIA - 01,2971874,2054760,Puglia,4052566,13,311735,5,810513,8.0,...,0.0,0.000000,7132.0,0.347096,831740.0,40.478693,56229.0,2.736524,1.186903,2.863741
19,SICILIA - 01,1873329,1155198,Sicilia,2393438,8,299179,3,797813,5.0,...,0.0,0.000000,0.0,0.000000,506530.0,43.847894,33981.0,2.941574,0.667245,2.859640
18,SARDEGNA - 01,1269860,834998,0,0,0,0,0,0,0.0,...,0.0,0.000000,4894.0,0.586109,326480.0,39.099495,22256.0,2.665396,1.427309,2.796174


# Export to Excel

In [24]:


with pd.ExcelWriter("plurinominali.xlsx") as writer:
    rep_camera.sort_values(by = 'COLLEGIOPLURINOMINALE 2022').to_excel(writer, sheet_name="Plurinominali Camera", index = False)  
    rep_senato.sort_values(by = 'COLLEGIOPLURINOMINALE 2022').to_excel(writer, sheet_name="Plurinominali Senato", index = False)  
    
    

In [173]:
senato[(senato['LISTA'] == 'POTERE AL POPOLO!') & 
       (senato['REGIONE'] == 'CAMPANIA')
      ]

,REGIONE,COLLEGIOPLURINOMINALE 2018,COLLEGIOUNINOMINALE 2018,COMUNE,ELETTORI,ELETTORI_MASCHI,VOTANTI,VOTANTI_MASCHI,SCHEDE_BIANCHE,COGNOME,NOME,DATA_NASCITA,LUOGO_NASCITA,SESSO,VOTI_CANDIDATO,LISTA,VOTI_LISTA,COLLEGIOPLURINOMINALE 2022
96576,CAMPANIA,CAMPANIA - 01,01 - BENEVENTO,AIROLA,5901,2851,4521,2289,96,FAPPIANO,GIUSEPPE,10/2/1956 00:00:00,CERRETO SANNITA (BN),M,37,POTERE AL POPOLO!,36.0,CAMPANIA - 02
96594,CAMPANIA,CAMPANIA - 01,01 - BENEVENTO,AMOROSI,2065,1005,1469,762,36,FAPPIANO,GIUSEPPE,10/2/1956 00:00:00,CERRETO SANNITA (BN),M,20,POTERE AL POPOLO!,9.0,CAMPANIA - 02
96612,CAMPANIA,CAMPANIA - 01,01 - BENEVENTO,APOLLOSA,1922,931,1506,767,15,FAPPIANO,GIUSEPPE,10/2/1956 00:00:00,CERRETO SANNITA (BN),M,24,POTERE AL POPOLO!,0.0,CAMPANIA - 02
96630,CAMPANIA,CAMPANIA - 01,01 - BENEVENTO,ARPAIA,1416,696,1001,515,11,FAPPIANO,GIUSEPPE,10/2/1956 00:00:00,CERRETO SANNITA (BN),M,1,POTERE AL POPOLO!,1.0,CAMPANIA - 02
96648,CAMPANIA,CAMPANIA - 01,01 - BENEVENTO,ARPAISE,605,307,440,239,7,FAPPIANO,GIUSEPPE,10/2/1956 00:00:00,CERRETO SANNITA (BN),M,9,POTERE AL POPOLO!,9.0,CAMPANIA - 02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106316,CAMPANIA,CAMPANIA - 03,11 - BATTIPAGLIA,TORTORELLA,407,198,260,143,15,CARAFFA,CLAUDIO,26/8/1957 00:00:00,SALERNO,M,0,POTERE AL POPOLO!,0.0,CAMPANIA - 02
106334,CAMPANIA,CAMPANIA - 03,11 - BATTIPAGLIA,TRENTINARA,1249,619,811,421,21,CARAFFA,CLAUDIO,26/8/1957 00:00:00,SALERNO,M,13,POTERE AL POPOLO!,13.0,CAMPANIA - 02
106352,CAMPANIA,CAMPANIA - 03,11 - BATTIPAGLIA,VALLE DELL'ANGELO,198,100,151,79,6,CARAFFA,CLAUDIO,26/8/1957 00:00:00,SALERNO,M,2,POTERE AL POPOLO!,2.0,CAMPANIA - 02
106370,CAMPANIA,CAMPANIA - 03,11 - BATTIPAGLIA,VALLO DELLA LUCANIA,6229,2964,4605,2283,103,CARAFFA,CLAUDIO,26/8/1957 00:00:00,SALERNO,M,58,POTERE AL POPOLO!,56.0,CAMPANIA - 02
